In [ ]:
1. Build a Flask app that scrapes data from multiple websites and displays it on your site.
You can try to scrap websites like youtube , amazon and show data on output pages and deploy it on cloud
platform .

In [ ]:
from flask import Flask, render_template, request, flash, redirect, url_for
from bs4 import BeautifulSoup
import requests
import uuid
import os
import pathlib
from urllib.parse import urljoin
from validators import url as validators

app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/scrape", methods=("GET", "POST"))
def scrape():
    global requested_url, specific_element, tag
    if request.method == "POST":
        requested_url = request.form.get('urltext')
        tag = request.form.get('specificElement')
        specific_element = []
        response = requests.get(requested_url)
        soup = BeautifulSoup(response.content, "html.parser")
        elements = soup.find_all(tag)
        for element in elements:
            specific_element.append(element.text)
        counter = len(specific_element)
        image_paths = image_handler(tag, specific_element, requested_url)
        return render_template("index.html",
            url=requested_url,
            counter=counter,
            image_paths=image_paths,
            results=specific_element
        )
    except Exception as e:
        flash(e, "danger")
    return redirect(url_for('home'))

def image_handler(tag, specific_element, requested_url):
    image_paths = []
    if tag == 'img':
        images = [img['src'] for img in specific_element]
        for i in specific_element:
            image_path = i.attrs['src']
            valid_imgpath = validators.url(image_path)
            if valid_imgpath == True:
                full_path = image_path
            else:
                full_path = urljoin(requested_url, image_path)
                image_paths.append(full_path)
    return image_paths

@app.route("/download", methods=("GET", "POST"))
def downloader():
    try:
        for img in image_handler(tag, specific_element, requested_url):
            image_url = img
            filename = str(uuid.uuid4())
            file_ext = pathlib.Path(image_url).suffix
            picture_filename = filename + file_ext
            downloads_path = str(pathlib.Path.home() / "Downloads")
            picture_path = os.path.join(downloads_path, picture_filename)
            # download image logic here
        flash("Images saved in your Downloads directory", "success")
    except Exception as e:
        flash(e, "danger")
    return redirect(url_for('index'))

if __name__ == "__main__":
    app.run(debug=True)

In [ ]:
2. Create a Flask app that consumes data from external APIs and displays it to users.
Try to find an public API which will give you a data and based on that call it and deploy it on cloud platform

In [ ]:
from flask import Flask, request, jsonify
import requests

app = Flask(__name__)

# Set the secret key to some random bytes. Keep this really secret!
app.secret_key = b'_5#y2L"F4Q8z\n\xec]/'

# Using the OpenWeatherMap API as an example
API_URL = "http://api.openweathermap.org/data/2.5/weather"
API_KEY = "YOUR_API_KEY_HERE"

@app.route('/')
def index():
    return "Welcome to the Weather App!"

@app.route('/weather', methods=['GET'])
def get_weather():
    city = request.args.get('city')
    if city:
        params = {'q': city, 'appid': API_KEY}
        response = requests.get(API_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            return jsonify({'weather': data['weather'][0]['description']})
        else:
            return jsonify({'error': 'City not found'}), 404
    else:
        return jsonify({'error': 'City parameter is required'}), 400

if __name__ == "__main__":
    app.run(debug=True)

In [ ]:
3. Implement OAuth2 authentication to allow users to log in using their Google or Facebook accounts.

In [ ]:
from flask import Flask, request, jsonify
from flask_login import LoginManager, UserMixin, login_user, logout_user
from flask_dance.consumer import OAuthConsumerMixin
from flask_dance.contrib.google import make_google_blueprint
from flask_dance.contrib.facebook import make_facebook_blueprint
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config["SECRET_KEY"] = "secret_key_here"
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///users.db"

db = SQLAlchemy(app)
login_manager = LoginManager()
login_manager.init_app(app)

class User(UserMixin, db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(250), unique=True)

class OAuth(OAuthConsumerMixin, db.Model):
    user_id = db.Column(db.Integer, db.ForeignKey(User.id))
    user = db.relationship(User)

google_blueprint = make_google_blueprint(
    client_id="your_google_client_id",
    client_secret="your_google_client_secret",
    scope=["email", "profile"]
)

facebook_blueprint = make_facebook_blueprint(
    client_id="your_facebook_client_id",
    client_secret="your_facebook_client_secret",
    scope=["email", "public_profile"]
)

app.register_blueprint(google_blueprint, url_prefix="/login/google")
app.register_blueprint(facebook_blueprint, url_prefix="/login/facebook")

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(user_id)

@app.route("/login/google")
def google_login():
    return google_blueprint.authorize(callback=url_for("authorized", _external=True))

@app.route("/login/facebook")
def facebook_login():
    return facebook_blueprint.authorize(callback=url_for("authorized", _external=True))

@app.route("/authorized")
def authorized():
    if google_blueprint.authorized:
        response = google_blueprint.session.get("/userinfo")
        user_info = response.json()
        username = user_info["email"]
        user = User.query.filter_by(username=username).first()
        if user is None:
            user = User(username=username)
            db.session.add(user)
            db.session.commit()
        login_user(user)
        return redirect(url_for("index"))
    elif facebook_blueprint.authorized:
        response = facebook_blueprint.session.get("/me")
        user_info = response.json()
        username = user_info["email"]
        user = User.query.filter_by(username=username).first()
        if user is None:
            user = User(username=username)
            db.session.add(user)
            db.session.commit()
        login_user(user)
        return redirect(url_for("index"))
    return "Login failed"

@app.route("/logout")
def logout():
    logout_user()
    return redirect(url_for("index"))

@app.route("/")
def index():
    return "Welcome to the app!"

if __name__ == "__main__":
    app.run(debug=True)

In [ ]:
4. Develop a recommendation system using Flask that suggests content to users based on their preferences.

In [ ]:
from flask import Flask, render_template, request
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

app = Flask(__name__)

# Load the dataset
netflix_titles_df = pd.read_csv('netflix_titles.csv')

# Preprocess the data
netflix_titles_df.drop(netflix_titles_df.columns[[0,1,5,6,7,9]], axis=1, inplace=True)
netflix_titles_df.count()

# Create a TF-IDF vectorizer
tfidf_vectorizer_params = TfidfVectorizer(lowercase=True, stop_words='english', ngram_range=(1, 3), max_df = .5)
titles_corpus = netflix_titles_df.apply(' '.join, axis=1)
tfidf_vectorizer = tfidf_vectorizer_params.fit_transform(titles_corpus)

# Create a cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_vectorizer, tfidf_vectorizer)

@app.route("/")
def home():
    return render_template('index.html')

@app.route("/recommend", methods=['POST'])
def recommend():
    movie_name = request.form['movies']
    idx = netflix_titles_df[netflix_titles_df['title'] == movie_name].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    recommend_movies = netflix_titles_df.iloc[movie_indices]
    return render_template('index.html', recommend_movies=recommend_movies, name=movie_name)

if __name__ == "__main__":
    app.run(debug=True, host='0.0.0.0')